<a href="https://colab.research.google.com/github/HJoon-K/Find_Dust/blob/yang/yjh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder

from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering

from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import silhouette_score

from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import linkage, fcluster

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
# 경고 메세지 감춤
import warnings
warnings.filterwarnings('ignore')

In [4]:
dust16 = pd.read_excel('data/기간별_일평균_대기환경_정보_2016년.xlsx')
dust16

,측정일자,권역명,측정소명,미세먼지(㎍/㎥),초미세먼지(㎍/㎥),오존(ppm),이산화질소농도(ppm),일산화탄소농도(ppm),아황산가스농도(ppm)
0,20160101,동북권,광진구,85,64,0.004,0.046,0.9,0.006
1,20160101,동남권,강남구,73,44,0.006,0.047,0.8,0.006
2,20160101,서남권,강서구,66,38,0.005,0.048,0.9,0.006
3,20160101,서남권,구로구,56,41,0.007,0.038,1.1,0.007
4,20160101,동북권,성북구,71,45,0.005,0.056,1.3,0.007
...,...,...,...,...,...,...,...,...,...
9143,20161231,서북권,마포구,56,47,0.003,0.062,1.2,0.005
9144,20161231,서남권,양천구,61,37,0.002,0.053,1.2,0.006
9145,20161231,동남권,강동구,63,41,0.003,0.045,0.8,0.004
9146,20161231,서남권,영등포구,56,36,0.004,0.036,1.3,0.005


In [5]:
dust16.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9148 entries, 0 to 9147
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   측정일자          9148 non-null   int64  
 1   권역명           9148 non-null   object 
 2   측정소명          9148 non-null   object 
 3   미세먼지(㎍/㎥)     9148 non-null   int64  
 4   초미세먼지(㎍/㎥)    9148 non-null   int64  
 5   오존(ppm)       9148 non-null   float64
 6   이산화질소농도(ppm)  9148 non-null   float64
 7   일산화탄소농도(ppm)  9148 non-null   float64
 8   아황산가스농도(ppm)  9148 non-null   float64
dtypes: float64(4), int64(3), object(2)
memory usage: 643.3+ KB


In [6]:
dust16.drop('권역명', axis=1, inplace=True)
dust16.drop('오존(ppm)', axis=1, inplace=True)
dust16.drop('이산화질소농도(ppm)', axis=1, inplace=True)
dust16.drop('일산화탄소농도(ppm)', axis=1, inplace=True)
dust16.drop('아황산가스농도(ppm)', axis=1, inplace=True)
dust16

,측정일자,측정소명,미세먼지(㎍/㎥),초미세먼지(㎍/㎥)
0,20160101,광진구,85,64
1,20160101,강남구,73,44
2,20160101,강서구,66,38
3,20160101,구로구,56,41
4,20160101,성북구,71,45
...,...,...,...,...
9143,20161231,마포구,56,47
9144,20161231,양천구,61,37
9145,20161231,강동구,63,41
9146,20161231,영등포구,56,36


In [7]:
dust16.columns = ['date', 'gu', 'PM10', 'PM25']
dust16

,date,gu,PM10,PM25
0,20160101,광진구,85,64
1,20160101,강남구,73,44
2,20160101,강서구,66,38
3,20160101,구로구,56,41
4,20160101,성북구,71,45
...,...,...,...,...
9143,20161231,마포구,56,47
9144,20161231,양천구,61,37
9145,20161231,강동구,63,41
9146,20161231,영등포구,56,36


In [8]:
dust16.to_csv('data/dust2016.csv', index=False)

In [9]:
dust2016 = pd.read_csv('data/dust2016.csv')
dust2016

,date,gu,PM10,PM25
0,20160101,광진구,85,64
1,20160101,강남구,73,44
2,20160101,강서구,66,38
3,20160101,구로구,56,41
4,20160101,성북구,71,45
...,...,...,...,...
9143,20161231,마포구,56,47
9144,20161231,양천구,61,37
9145,20161231,강동구,63,41
9146,20161231,영등포구,56,36


In [10]:
dust17 = pd.read_excel('data/기간별_일평균_대기환경_정보_2017년.xlsx')
dust17

,측정일자,권역명,측정소명,미세먼지(㎍/㎥),초미세먼지(㎍/㎥),오존(ppm),이산화질소농도(ppm),일산화탄소농도(ppm),아황산가스농도(ppm)
0,20170101,도심권,중구,81,64,0.003,0.057,1.0,0.005
1,20170101,도심권,용산구,77,73,0.003,0.039,1.1,0.005
2,20170101,도심권,종로구,82,67,0.003,0.054,1.0,0.004
3,20170101,서북권,서대문구,91,50,0.008,0.039,0.8,0.005
4,20170101,서북권,은평구,70,46,0.005,0.042,0.7,0.006
...,...,...,...,...,...,...,...,...,...
9120,20171231,서남권,금천구,63,38,0.019,0.025,0.6,0.005
9121,20171231,동남권,송파구,71,34,0.024,0.018,0.5,0.004
9122,20171231,동남권,강남구,54,38,0.016,0.020,0.6,0.006
9123,20171231,동남권,강동구,69,36,0.021,0.021,0.6,0.004


In [11]:
dust17.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9125 entries, 0 to 9124
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   측정일자          9125 non-null   int64  
 1   권역명           9125 non-null   object 
 2   측정소명          9125 non-null   object 
 3   미세먼지(㎍/㎥)     9125 non-null   int64  
 4   초미세먼지(㎍/㎥)    9125 non-null   int64  
 5   오존(ppm)       9125 non-null   float64
 6   이산화질소농도(ppm)  9125 non-null   float64
 7   일산화탄소농도(ppm)  9125 non-null   float64
 8   아황산가스농도(ppm)  9125 non-null   float64
dtypes: float64(4), int64(3), object(2)
memory usage: 641.7+ KB


In [12]:
dust17.drop('권역명', axis=1, inplace=True)
dust17.drop('오존(ppm)', axis=1, inplace=True)
dust17.drop('이산화질소농도(ppm)', axis=1, inplace=True)
dust17.drop('일산화탄소농도(ppm)', axis=1, inplace=True)
dust17.drop('아황산가스농도(ppm)', axis=1, inplace=True)
dust17

,측정일자,측정소명,미세먼지(㎍/㎥),초미세먼지(㎍/㎥)
0,20170101,중구,81,64
1,20170101,용산구,77,73
2,20170101,종로구,82,67
3,20170101,서대문구,91,50
4,20170101,은평구,70,46
...,...,...,...,...
9120,20171231,금천구,63,38
9121,20171231,송파구,71,34
9122,20171231,강남구,54,38
9123,20171231,강동구,69,36


In [13]:
dust17.columns = ['date', 'gu', 'PM10', 'PM25']
dust17

,date,gu,PM10,PM25
0,20170101,중구,81,64
1,20170101,용산구,77,73
2,20170101,종로구,82,67
3,20170101,서대문구,91,50
4,20170101,은평구,70,46
...,...,...,...,...
9120,20171231,금천구,63,38
9121,20171231,송파구,71,34
9122,20171231,강남구,54,38
9123,20171231,강동구,69,36


In [14]:
dust17.to_csv('data/dust2017.csv', index=False)

In [15]:
dust2017 = pd.read_csv('data/dust2017.csv')
dust2017

,date,gu,PM10,PM25
0,20170101,중구,81,64
1,20170101,용산구,77,73
2,20170101,종로구,82,67
3,20170101,서대문구,91,50
4,20170101,은평구,70,46
...,...,...,...,...
9120,20171231,금천구,63,38
9121,20171231,송파구,71,34
9122,20171231,강남구,54,38
9123,20171231,강동구,69,36


In [16]:
dust18 = pd.read_csv('data/기간별_일평균_대기환경_정보_2018년.csv', encoding='euc-kr')
dust18

,측정일자,권역코드,권역명,측정소코드,측정소명,미세먼지(㎍/㎥),초미세먼지(㎍/㎥),오존(ppm),이산화질소농도(ppm),일산화탄소농도(ppm),아황산가스농도(ppm)
0,20180101,100,도심권,111121,중구,32,18,0.015,0.028,0.5,0.005
1,20180101,104,동남권,111274,강동구,48,24,0.010,0.038,0.7,0.005
2,20180101,104,동남권,111273,송파구,52,23,0.013,0.028,0.7,0.004
3,20180101,104,동남권,111262,서초구,49,24,0.011,0.031,0.6,0.005
4,20180101,104,동남권,111261,강남구,34,22,0.010,0.033,0.6,0.006
...,...,...,...,...,...,...,...,...,...,...,...
9095,20181231,103,서남권,111221,구로구,46,23,0.008,0.041,0.5,0.006
9096,20181231,103,서남권,111231,영등포구,48,33,0.009,0.033,0.9,0.005
9097,20181231,103,서남권,111241,동작구,48,32,0.007,0.052,0.8,0.004
9098,20181231,103,서남권,111251,관악구,48,31,0.008,0.054,0.7,0.005


In [17]:
dust18.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9100 entries, 0 to 9099
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   측정일자          9100 non-null   int64  
 1   권역코드          9100 non-null   int64  
 2   권역명           9100 non-null   object 
 3   측정소코드         9100 non-null   int64  
 4   측정소명          9100 non-null   object 
 5   미세먼지(㎍/㎥)     9100 non-null   int64  
 6   초미세먼지(㎍/㎥)    9100 non-null   int64  
 7   오존(ppm)       9100 non-null   float64
 8   이산화질소농도(ppm)  9100 non-null   float64
 9   일산화탄소농도(ppm)  9100 non-null   float64
 10  아황산가스농도(ppm)  9100 non-null   float64
dtypes: float64(4), int64(5), object(2)
memory usage: 782.2+ KB


In [18]:
dust18.drop('권역코드', axis=1, inplace=True)
dust18.drop('권역명', axis=1, inplace=True)
dust18.drop('측정소코드', axis=1, inplace=True)
dust18.drop('오존(ppm)', axis=1, inplace=True)
dust18.drop('이산화질소농도(ppm)', axis=1, inplace=True)
dust18.drop('일산화탄소농도(ppm)', axis=1, inplace=True)
dust18.drop('아황산가스농도(ppm)', axis=1, inplace=True)
dust18

,측정일자,측정소명,미세먼지(㎍/㎥),초미세먼지(㎍/㎥)
0,20180101,중구,32,18
1,20180101,강동구,48,24
2,20180101,송파구,52,23
3,20180101,서초구,49,24
4,20180101,강남구,34,22
...,...,...,...,...
9095,20181231,구로구,46,23
9096,20181231,영등포구,48,33
9097,20181231,동작구,48,32
9098,20181231,관악구,48,31


In [19]:
dust18.columns = ['date', 'gu', 'PM10', 'PM25']
dust18

,date,gu,PM10,PM25
0,20180101,중구,32,18
1,20180101,강동구,48,24
2,20180101,송파구,52,23
3,20180101,서초구,49,24
4,20180101,강남구,34,22
...,...,...,...,...
9095,20181231,구로구,46,23
9096,20181231,영등포구,48,33
9097,20181231,동작구,48,32
9098,20181231,관악구,48,31


In [20]:
dust18.to_csv('data/dust2018.csv', index=False)

In [21]:
dust2018 = pd.read_csv('data/dust2018.csv')
dust2018

,date,gu,PM10,PM25
0,20180101,중구,32,18
1,20180101,강동구,48,24
2,20180101,송파구,52,23
3,20180101,서초구,49,24
4,20180101,강남구,34,22
...,...,...,...,...
9095,20181231,구로구,46,23
9096,20181231,영등포구,48,33
9097,20181231,동작구,48,32
9098,20181231,관악구,48,31


In [22]:
dust19 = pd.read_csv('data/기간별_일평균_대기환경_정보_2019년.csv', encoding='euc-kr')
dust19

,측정일자,권역코드,권역명,측정소코드,측정소명,미세먼지(㎍/㎥),초미세먼지(㎍/㎥),오존(ppm),이산화질소농도(ppm),일산화탄소농도(ppm),아황산가스농도(ppm)
0,20190101,100,도심권,111121,중구,38,26,0.019,0.032,0.6,0.003
1,20190101,104,동남권,111274,강동구,43,32,0.010,0.040,0.8,0.005
2,20190101,104,동남권,111273,송파구,44,28,0.011,0.042,0.8,0.004
3,20190101,104,동남권,111262,서초구,37,20,0.015,0.032,0.4,0.004
4,20190101,104,동남권,111261,강남구,37,31,0.012,0.028,0.6,0.005
...,...,...,...,...,...,...,...,...,...,...,...
8941,20191231,103,서남권,111212,강서구,29,16,0.024,0.015,0.4,0.004
8942,20191231,103,서남권,111221,구로구,20,14,0.027,0.014,0.2,0.003
8943,20191231,103,서남권,111231,영등포구,22,14,0.020,0.016,0.5,0.002
8944,20191231,103,서남권,111241,동작구,28,19,0.017,0.017,0.3,0.003


In [23]:
dust19.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8946 entries, 0 to 8945
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   측정일자          8946 non-null   int64  
 1   권역코드          8946 non-null   int64  
 2   권역명           8946 non-null   object 
 3   측정소코드         8946 non-null   int64  
 4   측정소명          8946 non-null   object 
 5   미세먼지(㎍/㎥)     8946 non-null   int64  
 6   초미세먼지(㎍/㎥)    8946 non-null   int64  
 7   오존(ppm)       8946 non-null   float64
 8   이산화질소농도(ppm)  8946 non-null   float64
 9   일산화탄소농도(ppm)  8946 non-null   float64
 10  아황산가스농도(ppm)  8946 non-null   float64
dtypes: float64(4), int64(5), object(2)
memory usage: 768.9+ KB


In [24]:
dust19.drop('권역코드', axis=1, inplace=True)
dust19.drop('권역명', axis=1, inplace=True)
dust19.drop('측정소코드', axis=1, inplace=True)
dust19.drop('오존(ppm)', axis=1, inplace=True)
dust19.drop('이산화질소농도(ppm)', axis=1, inplace=True)
dust19.drop('일산화탄소농도(ppm)', axis=1, inplace=True)
dust19.drop('아황산가스농도(ppm)', axis=1, inplace=True)
dust19

,측정일자,측정소명,미세먼지(㎍/㎥),초미세먼지(㎍/㎥)
0,20190101,중구,38,26
1,20190101,강동구,43,32
2,20190101,송파구,44,28
3,20190101,서초구,37,20
4,20190101,강남구,37,31
...,...,...,...,...
8941,20191231,강서구,29,16
8942,20191231,구로구,20,14
8943,20191231,영등포구,22,14
8944,20191231,동작구,28,19


In [25]:
dust19.columns = ['date', 'gu', 'PM10', 'PM25']
dust19

,date,gu,PM10,PM25
0,20190101,중구,38,26
1,20190101,강동구,43,32
2,20190101,송파구,44,28
3,20190101,서초구,37,20
4,20190101,강남구,37,31
...,...,...,...,...
8941,20191231,강서구,29,16
8942,20191231,구로구,20,14
8943,20191231,영등포구,22,14
8944,20191231,동작구,28,19


In [26]:
dust19.to_csv('data/dust2019.csv', index=False)

In [27]:
dust2019 = pd.read_csv('data/dust2019.csv')
dust2019

,date,gu,PM10,PM25
0,20190101,중구,38,26
1,20190101,강동구,43,32
2,20190101,송파구,44,28
3,20190101,서초구,37,20
4,20190101,강남구,37,31
...,...,...,...,...
8941,20191231,강서구,29,16
8942,20191231,구로구,20,14
8943,20191231,영등포구,22,14
8944,20191231,동작구,28,19


In [28]:
dust20 = pd.read_csv('data/기간별_일평균_대기환경_정보_2020년.csv', encoding='euc-kr')
dust20

,측정일자,권역코드,권역명,측정소코드,측정소명,미세먼지(㎍/㎥),오존(ppm),이산화질소농도(ppm),일산화탄소농도(ppm),아황산가스농도(ppm),초미세먼지(㎍/㎥)
0,20201231,100,도심권,111121,중구,29,0.020,0.016,0.4,0.003,13
1,20201231,100,도심권,111123,종로구,28,0.022,0.016,0.4,0.003,12
2,20201231,100,도심권,111131,용산구,34,0.014,0.019,0.4,0.003,10
3,20201231,101,서북권,111181,은평구,26,0.023,0.016,0.5,0.004,10
4,20201231,101,서북권,111191,서대문구,20,0.020,0.017,0.5,0.004,10
...,...,...,...,...,...,...,...,...,...,...,...
9095,20200101,103,서남권,111301,양천구,40,0.003,0.042,0.7,0.004,26
9096,20200101,104,동남권,111261,강남구,30,0.003,0.037,0.7,0.003,22
9097,20200101,104,동남권,111262,서초구,37,0.008,0.046,0.5,0.003,23
9098,20200101,104,동남권,111273,송파구,31,0.004,0.037,0.6,0.003,21


In [29]:
dust20.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9100 entries, 0 to 9099
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   측정일자          9100 non-null   int64  
 1   권역코드          9100 non-null   int64  
 2   권역명           9100 non-null   object 
 3   측정소코드         9100 non-null   int64  
 4   측정소명          9100 non-null   object 
 5   미세먼지(㎍/㎥)     9100 non-null   int64  
 6   오존(ppm)       9100 non-null   float64
 7   이산화질소농도(ppm)  9100 non-null   float64
 8   일산화탄소농도(ppm)  9100 non-null   float64
 9   아황산가스농도(ppm)  9100 non-null   float64
 10  초미세먼지(㎍/㎥)    9100 non-null   int64  
dtypes: float64(4), int64(5), object(2)
memory usage: 782.2+ KB


In [30]:
dust20.drop('권역코드', axis=1, inplace=True)
dust20.drop('권역명', axis=1, inplace=True)
dust20.drop('측정소코드', axis=1, inplace=True)
dust20.drop('오존(ppm)', axis=1, inplace=True)
dust20.drop('이산화질소농도(ppm)', axis=1, inplace=True)
dust20.drop('일산화탄소농도(ppm)', axis=1, inplace=True)
dust20.drop('아황산가스농도(ppm)', axis=1, inplace=True)
dust20

,측정일자,측정소명,미세먼지(㎍/㎥),초미세먼지(㎍/㎥)
0,20201231,중구,29,13
1,20201231,종로구,28,12
2,20201231,용산구,34,10
3,20201231,은평구,26,10
4,20201231,서대문구,20,10
...,...,...,...,...
9095,20200101,양천구,40,26
9096,20200101,강남구,30,22
9097,20200101,서초구,37,23
9098,20200101,송파구,31,21


In [31]:
dust20.columns = ['date', 'gu', 'PM10', 'PM25']
dust20

,date,gu,PM10,PM25
0,20201231,중구,29,13
1,20201231,종로구,28,12
2,20201231,용산구,34,10
3,20201231,은평구,26,10
4,20201231,서대문구,20,10
...,...,...,...,...
9095,20200101,양천구,40,26
9096,20200101,강남구,30,22
9097,20200101,서초구,37,23
9098,20200101,송파구,31,21


In [32]:
dust20.to_csv('data/dust2020.csv', index=False)

In [33]:
dust2020 = pd.read_csv('data/dust2020.csv')
dust2020

,date,gu,PM10,PM25
0,20201231,중구,29,13
1,20201231,종로구,28,12
2,20201231,용산구,34,10
3,20201231,은평구,26,10
4,20201231,서대문구,20,10
...,...,...,...,...
9095,20200101,양천구,40,26
9096,20200101,강남구,30,22
9097,20200101,서초구,37,23
9098,20200101,송파구,31,21


In [34]:
traffic = pd.read_excel('data/01월 서울시 교통량 조사자료(2022).xlsx')
traffic

,지점별 일자별 교통량 범례,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,구분,설명,표현 예시,예시 설명
1,일자,교통량 조사 일자,20181201,43435
2,요일,교통량 조사 요일\n(※ 공휴일은 '일'로 표시),토,토요일
3,지점명,교통량 조사 도로명(조사지점명),성산로(금화터널),조사지점의 도로명(지점명)
4,지점번호,"조사지점을 5개 영역(A,B,C,D,F)으로 구분하고 일련번호를 부여함\n- [A(...",A-01,도심 1번 지점
5,방향,유입 : 외곽에서 서울시청으로 들어오는 방향\n유출 : 시울시청에서 외곽으로 나가는 방향,유입/유출,NaN
6,구분,조사지점에서 가까운 교차로명으로 방향표시,봉원고가차도→독립문역,봉원고가차도에서 독립문역 방향의 교통량
7,시간대,1시간 단위를 표시,0시,0시~1시
8,교통량,1시간 교통량,809,809대/시


In [35]:
traffic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   지점별 일자별 교통량 범례  9 non-null      object
 1   Unnamed: 1      9 non-null      object
 2   Unnamed: 2      9 non-null      object
 3   Unnamed: 3      8 non-null      object
dtypes: object(4)
memory usage: 416.0+ bytes


In [36]:
strtree = pd.read_csv('data/서울시 가로수 위치정보 (좌표계_ ITRF2000).csv', encoding='euc-kr')
strtree

,고유번호,구명,품계수종,수고,관리등급,위치,환경영향,전정,도로길이,구번호,...,관리방안,수목고유번호,차선수,보호덮개,가로명,가치등급,품계퓽고,수목명,X 좌표,Y 좌표
0,20000031,중구,8.0,5.0,2.0,광희초등학교,0.0,NaN,0.0,2.0,...,0.0,2-다산로-가-0312,NaN,1.0,다산로,0.0,3.0,느티나무,201413.805,451774.530
1,20000032,중구,4.0,3.5,2.0,NaN,0.0,NaN,1150.0,2.0,...,0.0,2-돈화문로-가-00051,NaN,1.0,돈화문로,0.0,6.0,이팝나무,199369.423,451738.693
2,20000033,중구,4.0,3.5,2.0,NaN,0.0,NaN,1150.0,2.0,...,0.0,2-돈화문로-가-00077,NaN,1.0,돈화문로,0.0,6.0,이팝나무,199340.344,452056.363
3,20000035,중구,8.0,8.0,1.0,삼각동,0.0,NaN,0.0,2.0,...,0.0,2-청계천로-가-0152,NaN,1.0,청계천로,0.0,6.0,회화나무,198644.220,452093.710
4,20000036,중구,8.0,10.0,1.0,신한은행,0.0,NaN,0.0,2.0,...,0.0,2-태평로-가-0400,NaN,1.0,태평로,0.0,6.0,소나무,197773.066,451247.136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7740,20005511,중구,8.0,15.0,1.0,신한은행,0.0,NaN,0.0,2.0,...,0.0,2-남대문로-가-0194,NaN,1.0,남대문로,0.0,6.0,은행나무,198489.857,452080.422
7741,20005517,중구,8.0,23.0,1.0,배재빌딩,0.0,NaN,0.0,2.0,...,0.0,2-서소문로-가-0032,NaN,1.0,서소문로,0.0,12.0,은행나무,197670.680,451503.218
7742,20005526,중구,4.0,6.0,1.0,NaN,0.0,NaN,0.0,2.0,...,0.0,2-청계천로-가-0033,NaN,1.0,청계천로,0.0,12.0,느티나무,200048.538,452183.948
7743,20003370,중구,8.0,16.0,1.0,방산시장,0.0,NaN,0.0,2.0,...,0.0,2-훈련원로-가-0221,NaN,1.0,훈련원로,0.0,6.0,은행나무,200176.060,452085.028


In [37]:
strtree.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7745 entries, 0 to 7744
Data columns (total 31 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   고유번호      7745 non-null   int64  
 1   구명        7745 non-null   object 
 2   품계수종      7730 non-null   float64
 3   수고        7730 non-null   float64
 4   관리등급      7730 non-null   float64
 5   위치        6450 non-null   object 
 6   환경영향      7730 non-null   float64
 7   전정        288 non-null    float64
 8   도로길이      7730 non-null   float64
 9   구번호       7724 non-null   float64
 10  전선유무      7730 non-null   float64
 11  순번        7730 non-null   float64
 12  품계활력      7730 non-null   float64
 13  흉고지름      7730 non-null   float64
 14  품계위치      7730 non-null   float64
 15  수관너비      7729 non-null   float64
 16  현존유무      7730 non-null   float64
 17  관리기관      7730 non-null   object 
 18  피해정도      6460 non-null   float64
 19  주변토지사용상태  7730 non-null   float64
 20  방재        3386 non-null   obje

In [38]:
district = pd.read_excel('data/행정구역(2016-2021)_20220901150113.xls')
district

,자치구별(1),자치구별(2),2017,2017.1,2017.2,2017.3,2017.4,2017.5,2018,2018.1,...,2020.2,2020.3,2020.4,2020.5,2021,2021.1,2021.2,2021.3,2021.4,2021.5
0,자치구별(1),자치구별(2),면적,면적,동,동,통 (개),반 (개),면적,면적,...,동,동,통 (개),반 (개),면적,면적,동,동,통 (개),반 (개)
1,자치구별(1),자치구별(2),면적 (m2),구성비 (%),행정 (개),법정 (개),소계,소계,면적 (m2),구성비 (%),...,행정 (개),법정 (개),소계,소계,면적 (m2),구성비 (%),행정 (개),법정 (개),소계,소계
2,서울시,소계,605.24,100,424,467,12622,94651,605.24,100,...,425,467,12764,95940,605.24,100,426,466,12859,96471
3,NaN,종로구,23.91,3.95,17,87,278,1527,23.91,3.95,...,17,87,279,1475,23.91,3.95,17,87,279,1475
4,NaN,중구,9.96,1.65,15,74,255,1612,9.96,1.65,...,15,74,253,1580,9.96,1.65,15,74,253,1580
5,NaN,용산구,21.87,3.61,16,36,351,2552,21.87,3.61,...,16,36,354,2581,21.87,3.61,16,36,356,2597
6,NaN,성동구,16.86,2.79,17,17,467,3695,16.86,2.79,...,17,17,467,3626,16.86,2.79,17,17,471,3645
7,NaN,광진구,17.06,2.82,15,7,361,2995,17.06,2.82,...,15,7,363,2999,17.06,2.82,15,7,370,3012
8,NaN,동대문구,14.22,2.35,14,10,359,2623,14.22,2.35,...,14,10,360,2633,14.22,2.35,14,10,360,2631
9,NaN,중랑구,18.5,3.06,16,6,545,4165,18.5,3.06,...,16,6,550,4165,18.5,3.06,16,6,558,4241


In [39]:
district.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 32 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   자치구별(1)  3 non-null      object
 1   자치구별(2)  28 non-null     object
 2   2017     28 non-null     object
 3   2017.1   28 non-null     object
 4   2017.2   28 non-null     object
 5   2017.3   28 non-null     object
 6   2017.4   28 non-null     object
 7   2017.5   28 non-null     object
 8   2018     28 non-null     object
 9   2018.1   28 non-null     object
 10  2018.2   28 non-null     object
 11  2018.3   28 non-null     object
 12  2018.4   28 non-null     object
 13  2018.5   28 non-null     object
 14  2019     28 non-null     object
 15  2019.1   28 non-null     object
 16  2019.2   28 non-null     object
 17  2019.3   28 non-null     object
 18  2019.4   28 non-null     object
 19  2019.5   28 non-null     object
 20  2020     28 non-null     object
 21  2020.1   28 non-null     object
 22  2020